## 以sklearn_neighbors提供的基于内存KNN为基准 
## 计算Recall % Overall Ratio

In [3]:
"""
使用sklearn_neighbors计算出查询负载的精确答案
保存精确KNN的结果索引--->后续用于计算Recall
保存精确KNN的求和距离--->后续用于计算Overall Ratio
"""
# 用sklearn.neighbors.NearestNeighbors进行精确KNN
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# 设置参数
K = 10  # 设置你想要的最近邻数

# 加载总向量数据集和查询负载向量
data_vectors = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Original_data\\128_100k_ResNet50_vector.npy')
query_vectors = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\sampled_100_vectors.npy')

# 初始化 NearestNeighbors 模型
# 使用 brute-force 方法，以确保得到精确的最近邻结果
neigh = NearestNeighbors(n_neighbors=K, algorithm='brute', metric='euclidean')
neigh.fit(data_vectors)

# 存储结果的列表
knn_indices = []  # 存储每个查询的最近邻索引
knn_distances = []  # 存储每个查询的最近邻距离

# 记录开始时间
start_time = time.time()

# 对查询集进行精确 KNN 搜索
distances, indices = neigh.kneighbors(query_vectors)

# 记录结束时间
end_time = time.time()

# 将结果保存到列表中
knn_indices = indices.tolist()  # 转换为 Python 列表格式
knn_distances = distances.tolist()  # 转换为 Python 列表格式

# 计算并打印总距离和
total_distance_sum = np.sum(distances)
print("总的距离之和:", total_distance_sum)

# 打印每个查询的运行时间（单位：毫秒）
elapsed_time = (end_time - start_time) * 1000  # 转换为毫秒
print(f"总运行时间: {elapsed_time:.4f} 毫秒")

# 打印每一个查询的精确 KNN 结果索引
# for i, (index_list, distance_list) in enumerate(zip(knn_indices, knn_distances)):
#     print(f"查询 {i+1} 的精确 KNN 结果索引:", index_list)
#     print(f"查询 {i+1} 的精确 KNN 结果距离:", distance_list)

# 保存 KNN 结果的索引和距离到 .npy 文件
np.save('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\sklearn_knn_indices.npy', knn_indices)
np.save('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\sklearn_knn_distances.npy', knn_distances)

print("精确 KNN 结果的索引和距离已成功保存！")

总的距离之和: 1987.5226485496926
总运行时间: 23.5772 毫秒
精确 KNN 结果的索引和距离已成功保存！


In [4]:
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# 加载数据集和查询集
data_vectors = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Original_data\\128_100k_ResNet50_vector.npy')
query_vectors = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\sampled_100_vectors.npy')

# 加载各个索引方法的结果索引
ids_list_HNSW = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\ids_list_HNSW.npy', allow_pickle=True)
ids_list_IVF_PQ = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\ids_list_IVF_PQ.npy', allow_pickle=True)
ids_list_right = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\ids_list_right.npy', allow_pickle=True)
knn_indices = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\sklearn_knn_indices.npy', allow_pickle=True)
ids_list_RESH = np.load('D:\\Python_Project\\Learned_Index\\Milvus\\RESH\\Query\\Completely_random\\sklearn_knn_indices.npy', allow_pickle=True)

# 使用 NearestNeighbors 计算每种索引方法的距离和
def calculate_total_distance_sum(query_vectors, data_vectors, indices_list):
    total_distance_sum = 0
    for i, query in enumerate(query_vectors):
        # 根据索引获取最近邻向量
        nearest_vectors = data_vectors[indices_list[i]]
        
        # 使用 NearestNeighbors 来计算查询与近邻的精确距离
        neigh = NearestNeighbors(n_neighbors=len(nearest_vectors), algorithm='brute', metric='euclidean')
        neigh.fit(nearest_vectors)
        
        # 获取距离和
        distances, _ = neigh.kneighbors([query])
        total_distance_sum += np.sum(distances[0])
    return total_distance_sum

# 计算召回率
def calculate_recall(predicted_indices, true_indices, K):
    total_recall = 0
    num_queries = len(predicted_indices)

    for i in range(num_queries):
        # 将基准 KNN 和待评估 KNN 的索引转换为集合
        true_set = set(true_indices[i])  # 基准结果
        predicted_set = set(predicted_indices[i])  # 待评估结果

        # 计算找到的正确结果数量（交集的大小）
        correct_results = len(true_set & predicted_set)

        # 计算每个查询的召回率
        recall = correct_results / K
        total_recall += recall

    # 计算平均召回率
    average_recall = total_recall / num_queries
    return average_recall

# 计算精确 KNN 的总距离和（基准）
total_distance_sum_exact = calculate_total_distance_sum(query_vectors, data_vectors, knn_indices)

# 计算 HNSW、IVF_PQ、FLAT、RESH 的总距离和
total_distance_sum_HNSW = calculate_total_distance_sum(query_vectors, data_vectors, ids_list_HNSW)
total_distance_sum_IVF_PQ = calculate_total_distance_sum(query_vectors, data_vectors, ids_list_IVF_PQ)
total_distance_sum_right = calculate_total_distance_sum(query_vectors, data_vectors, ids_list_right)
total_distance_sum_RESH = calculate_total_distance_sum(query_vectors, data_vectors, ids_list_RESH)

# 计算 Overall Ratio
overall_ratio_HNSW = total_distance_sum_HNSW / total_distance_sum_exact
overall_ratio_IVF_PQ = total_distance_sum_IVF_PQ / total_distance_sum_exact
overall_ratio_right = total_distance_sum_right / total_distance_sum_exact
overall_ratio_RESH = total_distance_sum_RESH / total_distance_sum_exact

# 计算召回率
recall_HNSW = calculate_recall(ids_list_HNSW, knn_indices, K)
recall_IVF_PQ = calculate_recall(ids_list_IVF_PQ, knn_indices, K)
recall_right = calculate_recall(ids_list_right, knn_indices, K)
recall_RESH = calculate_recall(ids_list_RESH, knn_indices, K)

# 打印结果
print("HNSW 的总距离和:", total_distance_sum_HNSW)
print("IVF_PQ 的总距离和:", total_distance_sum_IVF_PQ)
print("FLAT 的总距离和:", total_distance_sum_right)
print("RESH 的总距离和:", total_distance_sum_RESH)
print("精确 KNN 的总距离和（基准）:", total_distance_sum_exact)
print()
print("HNSW 的 Overall Ratio:", overall_ratio_HNSW)
print("IVF_PQ 的 Overall Ratio:", overall_ratio_IVF_PQ)
print("FLAT 的 Overall Ratio:", overall_ratio_right)
print("RESH 的 Overall Ratio:", overall_ratio_RESH)
print()
print("HNSW 的召回率:", recall_HNSW)
print("IVF_PQ 的召回率:", recall_IVF_PQ)
print("FLAT 的召回率:", recall_right)
print("RESH 的召回率:", recall_RESH)


HNSW 的总距离和: 11572.652153015744
IVF_PQ 的总距离和: 11572.919057391236
FLAT 的总距离和: 11575.592268089176
RESH 的总距离和: 1987.5226444513855
精确 KNN 的总距离和（基准）: 1987.5226444513855

HNSW 的 Overall Ratio: 5.822651724408471
IVF_PQ 的 Overall Ratio: 5.822786014388129
FLAT 的 Overall Ratio: 5.82413101073592
RESH 的 Overall Ratio: 1.0

HNSW 的召回率: 0.995
IVF_PQ 的召回率: 0.992
FLAT 的召回率: 0.995
RESH 的召回率: 1.0
